In [38]:
import networkx as nx
import numpy as np

### TWICE AROUND THE TREE

In [39]:
def findPathWeight(A, path):
    weight = 0
    for i in range(len(path) - 1):
        weight += A[path[i]][path[i + 1]]['weight']
    return weight

def twiceAroundTheTreeTSP(A):
    MST = nx.minimum_spanning_tree(A)
    path = nx.dfs_preorder_nodes(MST, 0)
    path = list(path)
    path.append(path[0])
    weight = findPathWeight(A, path)
    return weight, path

### CHRISTOFIDES

In [40]:
def findShortcutPath(A):
    path = list(nx.eulerian_circuit(A, 0))
    path = [x[0] for x in path]

    # remove duplicates
    shortcutPath = list(dict.fromkeys(path))
    
    return shortcutPath + [shortcutPath[0]]

def christofidesTSP(A):
    MST = nx.minimum_spanning_tree(A)
    degrees = nx.degree(MST)
    oddNodes = [x[0] for x in degrees if degrees[x[0]] % 2 == 1]
    oddNodesSubgraph = nx.subgraph(A, oddNodes)
    matching = list(nx.min_weight_matching(oddNodesSubgraph))

    MSTMultiGraph = nx.MultiGraph(MST)
    for i in range(len(matching)):
        node1 = matching[i][0]
        node2 = matching[i][1]
        MSTMultiGraph.add_edge(node1, node2, weight=A[node1][node2]['weight'])

    path = findShortcutPath(MSTMultiGraph)
    weight = findPathWeight(A, path)

    return weight, path

#### BRANCH AND BOUND

In [41]:
class Node:
    def __init__(self, bound, boundEdges, cost, solution):
        self.bound = bound
        self.boundEdges = boundEdges
        self.cost = cost
        self.solution = solution
    
    def __lt__(self, other):
        if len(self.solution) == len(other.solution):
            return self.bound < other.bound
        return len(self.solution) > len(other.solution)

def findTwoMinimalEdges(list):
    min1 = np.inf
    min2 = np.inf
    for j in list:
        if list[j]['weight'] < min1:
            min2 = min1
            min1 = list[j]['weight']
        elif list[j]['weight'] < min2:
            min2 = list[j]['weight']
    return min1, min2

def findInitialBound(A):
    bound = 0
    initialBoundEdges = np.zeros((A.number_of_nodes(), 2), dtype=list)
    for i in range(A.number_of_nodes()):
        min1, min2 = findTwoMinimalEdges(A[i])
        initialBoundEdges[i][0] = min1
        initialBoundEdges[i][1] = min2
        bound += min1 + min2
    return bound / 2, initialBoundEdges

def findBound(A, solution, boundEdges, bound):
    changedEdges = np.zeros(A.number_of_nodes(), dtype=int)
    newEdges = np.array(boundEdges)
    edgeWeight = A[solution[-2]][solution[-1]]['weight']
    sum = bound * 2
    if newEdges[solution[-2]][0] != edgeWeight:
        if changedEdges[solution[-2]] == 0:
            sum -= newEdges[solution[-2]][1]
            sum += edgeWeight
        else:
            sum -= newEdges[solution[-2]][0]
            sum += edgeWeight
        changedEdges[solution[-2]] += 1
    if newEdges[solution[-1]][0] != edgeWeight:
        if changedEdges[solution[-1]] == 0:
            sum -= newEdges[solution[-1]][1]
            sum += edgeWeight
        else:
            sum -= newEdges[solution[-1]][0]
            sum += edgeWeight
        changedEdges[solution[-1]] += 1
    return sum / 2, newEdges

In [42]:
from heapq import heappush, heappop

def branchAndBoundTSP(A):
    initialBound, initialBoundEdges = findInitialBound(A)
    root = Node(initialBound, initialBoundEdges, 0, [0])
    heap = []
    heappush(heap, root)
    best = np.inf
    solution = []
    nodeCount = 0
    while heap:
        node = heappop(heap)
        nodeCount += 1
        level = len(node.solution)
        if level > A.number_of_nodes():
            if best > node.cost:
                best = node.cost
                solution = node.solution
        else:
            if node.bound < best:
                if level < A.number_of_nodes() - 2:
                    for k in range(1, A.number_of_nodes()):
                        if k == node.solution[-1] or k == 0:
                            continue
                        edgeWeight = A[node.solution[-1]][k]['weight']
                        newBound, newEdges = findBound(A, node.solution + [k], node.boundEdges, node.bound) 
                        if k not in node.solution and newBound < best:
                            newNode = Node(newBound, newEdges, node.cost + edgeWeight, node.solution + [k])
                            if k == 2:
                                if 1 not in node.solution:  
                                    continue 
                            heappush(heap, newNode)
                else:
                    for k in range(1, A.number_of_nodes()):
                        if k == node.solution[-1] or k == 0:
                            continue
                        lastNode = 0
                        for i in range(1, A.number_of_nodes()):
                            if i not in node.solution + [k] and k != i:
                                lastNode = i
                                break
                        edgeWeight = A[node.solution[-1]][k]['weight']
                        nextEdgeWeight = A[k][lastNode]['weight']
                        lastEdgeWeight = A[lastNode][0]['weight']
                        cost = node.cost + edgeWeight + nextEdgeWeight + lastEdgeWeight
                        if k not in node.solution and cost < best:
                            newNode = Node(cost, [], cost, node.solution + [k, lastNode, 0])
                            heappush(heap, newNode)
    return best, solution



#### Teste

In [43]:
A = np.array([[0, 3, 1, 5, 8],
    [3, 0, 6, 7, 9],
    [1, 6, 0, 4, 2],
    [5, 7, 4, 0, 3],
    [8, 9, 2, 3, 0]])

A = nx.from_numpy_array(np.matrix(A), create_using=nx.Graph)

path, weight = twiceAroundTheTreeTSP(A)
print("Twice Around the Tree TSP")
print("Path: ", path)
print("Weight: ", weight)
print()

path, weight = christofidesTSP(A)
print("Christofides TSP")
print("Path: ", path)
print("Weight: ", weight)
print()

path, weight = branchAndBoundTSP(A)
#print("Branch and Bound TSP")
#print("Path: ", path)
#print("Weight: ", weight)
#print()


Twice Around the Tree TSP
Path:  16
Weight:  [0, 2, 4, 3, 1, 0]

Christofides TSP
Path:  16
Weight:  [0, 1, 3, 4, 2, 0]



In [44]:
G = np.array([[0, 4, 8, 9, 12],
    [4, 0, 6, 8, 9],
    [8, 6, 0, 10, 11],
    [9, 8, 10, 0, 7],   
    [12, 9, 11, 7, 0]])

G = nx.from_numpy_array(np.matrix(G), create_using=nx.Graph)

path, weight = twiceAroundTheTreeTSP(G)
print("Twice Around the Tree TSP")
print("Path: ", path)
print("Weight: ", weight)
print()

path, weight = christofidesTSP(G)
print("Christofides TSP")
print("Path: ", path)
print("Weight: ", weight)
print()

path, weight = branchAndBoundTSP(G)
print("Branch and Bound TSP")
print("Path: ", path)
print("Weight: ", weight)



Twice Around the Tree TSP
Path:  39
Weight:  [0, 1, 2, 3, 4, 0]

Christofides TSP
Path:  38
Weight:  [0, 1, 3, 4, 2, 0]

Branch and Bound TSP
Path:  37
Weight:  [0, 1, 2, 4, 3, 0]


In [45]:
#random test
n = 15
G = nx.complete_graph(n)
for (u, v, w) in G.edges(data=True):
    w['weight'] = np.random.randint(1, 10)
path, weight = twiceAroundTheTreeTSP(G)
print("Twice Around The Tree TSP")
print("Path: ", path)
print("Weight: ", weight)
print()

path, weight = christofidesTSP(G)
print("Christofides TSP")
print("Path: ", path)
print("Weight: ", weight)

path, weight = branchAndBoundTSP(G)
print("Branch and Bound TSP")
print("Path: ", path)
print("Weight: ", weight)


Twice Around The Tree TSP
Path:  39
Weight:  [0, 1, 10, 12, 13, 6, 2, 3, 9, 4, 5, 14, 7, 11, 8, 0]

Christofides TSP
Path:  38
Weight:  [0, 2, 8, 11, 7, 14, 4, 5, 9, 3, 12, 1, 13, 6, 10, 0]
Branch and Bound TSP
Path:  21
Weight:  [0, 1, 10, 13, 6, 5, 4, 9, 3, 11, 7, 14, 12, 8, 2, 0]


In [46]:
import time
import numpy as np
from memory_profiler import memory_usage

def execute_algorithm(algorithm, data):
    start_time = time.time()
    max_time = 30 * 60  # 30 minutos em segundos

    try:
        # Executa o algoritmo e mede o uso de memória
        mem_usage = memory_usage((algorithm, (data,)))
        result = algorithm(data)
        end_time = time.time()

        execution_time = end_time - start_time
        if execution_time > max_time:
            raise TimeoutError
        
        print("Algorithm: ", algorithm.__name__)
        print("Weight: ", result[0])
        print("Path: ", result[1])
        print("Execution time: ", execution_time)
        print("Memory usage: ", np.max(mem_usage))
        print()

        return result, execution_time, np.max(mem_usage)
    except TimeoutError:
        return "NA", "NA", "NA"

execute_algorithm(twiceAroundTheTreeTSP, G)
execute_algorithm(christofidesTSP, G)
execute_algorithm(branchAndBoundTSP, G)


Algorithm:  twiceAroundTheTreeTSP
Weight:  39
Path:  [0, 1, 10, 12, 13, 6, 2, 3, 9, 4, 5, 14, 7, 11, 8, 0]
Execution time:  0.06387829780578613
Memory usage:  103.4375

Algorithm:  christofidesTSP
Weight:  38
Path:  [0, 2, 8, 11, 7, 14, 4, 5, 9, 3, 12, 1, 13, 6, 10, 0]
Execution time:  0.04223132133483887
Memory usage:  103.4375

Algorithm:  branchAndBoundTSP
Weight:  21
Path:  [0, 1, 10, 13, 6, 5, 4, 9, 3, 11, 7, 14, 12, 8, 2, 0]
Execution time:  1.4978086948394775
Memory usage:  103.45703125



((21, [0, 1, 10, 13, 6, 5, 4, 9, 3, 11, 7, 14, 12, 8, 2, 0]),
 1.4978086948394775,
 103.45703125)